## Base - run all of this section before any other sections

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
C = 29979245800.0 # speed of light, in [cm s^-1]
M_HI = 1.6735575e-24 # mass of Hydrogen, in [gram]
M_p = 1.67262193269e-24 # mass of proton, in [gram]
k = 1.380649e-16 # Boltzmann constant, in [g cm^2 s^-2 K^-1]
M_PI = 3.14159265358979323846 # pi
T_2p = 1.6e-9 # lifetime of 1s to 2p, in [s]
v_center = 2.47e15 # the center frequency of Lyman alpha emission, in [Hz]
A = 6.25e8 # spontaneous decay rate, in [s^-1]
DNHI = 2.5e16 # column density of Hydrogen atoms, in [cm^-2]

NGRID = 2000 # number of grid
N_NU = 128 # number of frequency bins

In [ ]:
def count(li, min_, max_):
    ctr = 0
    for x in li:
        if min_ <= x <= max_:
            ctr += 1
    return ctr

In [ ]:
def fac(n):
    fac = 1
    for i in range(1, int(n)+1):
        fac = fac * i
    return fac

def F(v):
    if abs(v) <=8:
        f = v + v**3/3.
        for n in range(2, int(v*v)+int(abs(v)*10)):
            f += v**(2 * n + 1) / (fac(n) * (2 * n + 1))
        F = math.exp(-v * v) * f
    else:
        F = .5*(1/v + .5*v**-3 + (3/4 )* v**-5 + (15/8)*v**-7 + (105/16)*v**-9 +(945/32)*v**-11 +(10395/64)*v**-13 
                 +(135135/128)*v**-15 + (2027025/256)*v**-17 + (34459425/512)*v**-19 + (654729075/1024)*v**-21 
                 + (13749310575/2048)*v**-23 + (316234143225/4096)*v**-25 + (7905853580625/8192)*v**-27)
    return F

def H_xa(x, a):
    H_0 = math.exp(-x * x)
    H_1 = -2 / math.sqrt(math.pi) * (1 - 2 * x * F(x))
    H_2 = (1 - 2 * x * x) * math.exp(-x * x)
    
    H_x = H_0 + a * H_1 + a * a * H_2
    H_x = H_x / math.sqrt(math.pi)
    
    return H_x

In [ ]:
def get_H_x(L_v, THI):
    lambda_i = C / v_center
    lambda_d = math.sqrt(2 * k * THI / M_p) * lambda_i / C
    x = (C / (L_v + v_center) - lambda_i) / lambda_d
    a = lambda_i * lambda_i / (4 * M_PI * C * lambda_d * T_2p)
    H_x = H_xa(x, a)
    
    return x, a, H_x

## Generator

In [ ]:
# initial position

# read file - change the file name to your corresponding file
with open('C:/data/ly_position.txt') as f:
    f = [x.strip() for x in f if x.strip()]
    data = [tuple(map(float,x.split())) for x in f[0:]]
    position_i = [x[0] for x in data]
    
print('The max value is', max(position_i), ', the min value is', min(position_i), 'and the mean is', np.mean(position_i))

# plot
n = int(max(position_i) - min(position_i))
plt.hist(position_i, bins = n, density = True)
plt.title('Generated position distribution')
plt.xlabel('Position(z)')
plt.ylabel('Probability density')
plt.show()

In [ ]:
# initial frequency with initial setting THI

# read file - change the file name to your corresponding file
with open('C:/Data/ly_frequency.txt') as f:
    f = [x.strip() for x in f if x.strip()]
    data = [tuple(map(float,x.split())) for x in f[0:]]
    frequency_i = [x[0] for x in data]
    
print('The max value is', max(frequency_i), ', the min value is', min(frequency_i), 'and the mean is', np.mean(frequency_i))

# plot -  change the plot size if needed
fig, ax = plt.subplots(figsize=(20, 10))
ax.hist(frequency_i, histtype='step', bins=200000, density=True, linewidth=2)
ax.set_xscale('symlog')
ax.set_xticks([-1e15, -1e12, -1e9, -1e6, -1e3, 0, 1e3, 1e6, 1e9, 1e12, 1e15])
ax.tick_params(axis='both', which='both', labelsize=25, width=1.2)
plt.title('Generated frequency distribution', size=25)
plt.xlabel('Frequency(Hz)', size=25)
plt.ylabel('Prbability density', size=25)
plt.show()

## Propagate

In [ ]:
# calculate the expected value with corresponding settings
def propagate(dv, mu, THI, y1H1, U):
    lambda_i = C / v_center
    lambda_d = math.sqrt(2 * k * THI / M_p) * lambda_i / C
    x = (C / (dv + v_center) - lambda_i) / lambda_d
    a = lambda_i * lambda_i / (4 * M_PI * C * lambda_d * T_2p)
    H_x = H_xa(x, a)

    sigma_tot = C / (math.sqrt(2 * k * THI / M_p) * v_center) / math.sqrt(M_PI) * 3 * A * C * C * H_x / (8 * M_PI * (dv + v_center) * (dv + v_center))

    l_mfp = 1 / (y1H1 * sigma_tot)
    dz = l_mfp * mu - U * l_mfp / C
    length = DNHI / y1H1

    return (dz / length)

In [ ]:
# change to the corresponding settings
expt = propagate(1e10, 0.5, 20000, 1.37e-4, 5e8)
print('The exptation value should be', expt)

# read file - change the file name to your corresponding file
with open('C:/Data/test_pro.txt') as f:
    f = [x.strip() for x in f if x.strip()]
    data = [tuple(map(float,x.split())) for x in f[0:]]
    data_propagate = [x[0] for x in data]
            
mean = np.mean(data_propagate)
sigma = np.std(data_propagate)

print('The mean is', mean)
print('The sigma is', sigma)

# plot
n_bins = 500
x = data_propagate

fig, ax = plt.subplots(figsize=(8, 4))

# plot the cumulative histogram - change the label to corresponding settings
n, bins, patches = ax.hist(x, n_bins, density=True, label='dv = 1e10, mu = 0.5, THI = 20000')

# tidy up the figure
ax.legend(loc='right')
ax.set_title('Propagating distance distribution')
ax.set_xlabel('Propagating distance(dz)')
ax.set_ylabel('Likelihood of occurrence')

# plot the expected distribution
def f(x):
    return expt * math.log(1./x)
f2 = np.vectorize(f)
x = np.arange(0.00001, 1, 0.00001)
plt.hist(f2(x), n_bins, density=True, histtype='step')

plt.show()

## Scatter

In [ ]:
# basic settings
u_1a = -7
u_1b = x - 0.25
u_1c = x + 0.25
u_1d = 7

if u_1b < u_1a:
    u_1b = u_1a
if u_1b > u_1d:
    u_1b = u_1d
if u_1c < u_1a:
    u_1c = u_1a
if u_1c > u_1d:
    u_1c = u_1d

In [ ]:
# scattering frequency

# read file - change the file name to your corresponding file
with open('C:/Data/test_2.txt') as f:
    f = [x.strip() for x in f if x.strip()]
    data = [tuple(map(float,x.split())) for x in f[0:]]
    u1 = [x[0] for x in data]
    count_A = [x[1] for x in data]
    count_B = [x[2] for x in data]
    count_C = [x[3] for x in data]
    
x1 = u1

print('larger than 3 =', count(x1, 3, max(x1)))
print('larger than x1 - 3 * a =', count(x1, 3 - 3 * a, max(x1)))

print('The number between u1a and u1b is', count(x1, u_1a, u_1b))
print('The total number generated between area_A is', sum(count_A))

print('The number between u1b and u1c is', count(x1, u_1b, u_1c))
print('The total number generated between area_B is', sum(count_B))

print('The number between u1c and u1d is', count(x1, u_1c, u_1d))
print('The total number generated between area_C is', sum(count_C))

# plot the cumulative histogram
fig, ax = plt.subplots(figsize=(20, 10))
ax.hist(x1, histtype='step', bins=10000, density=True, linewidth=2)
ax.set_xscale('symlog')
ax.tick_params(axis='both', which='both', labelsize=25, width=1.2)

# plot the expected distribution - change to your corresponding initial settings
x, a, H_x = get_H_x(-5e11, 13000)
def p(u1):
    return 1 / H_x * (a / M_PI**(3/2.) * math.exp(-u1**2) / (a**2 + (x - u1)**2))
p2 = np.vectorize(p)
u1 = np.arange(-5, 5, 0.0001)
plt.plot(u1, p2(u1))

plt.title('Scattering frequency (Middle)', size=25)
plt.xlabel('u1', size=25)
plt.ylabel('Probability density', size=25)
plt.show()

# plot the L-Tail - comment out this section if you do not need

n_bin = np.arange(3, 5, 0.01)

fig, ax = plt.subplots(figsize=(20, 10))
ax.hist(x1, histtype='step', bins=n_bin, linewidth=2)
ax.set_xscale('symlog')
ax.tick_params(axis='both', which='both', labelsize=25, width=1.2)
plt.show()

In [ ]:
# scattering direction and polarization

# read file - change the file name to your corresponding file
with open('C:/Data/ly_x_8p63_mup_00.txt') as f:
    f = [x.strip() for x in f if x.strip()]
    data = [tuple(map(float,x.split())) for x in f[0:]]
    data_direction = [x[0] for x in data]
    data_polarization = [x[1] for x in data]
    
mean_mu = np.mean(data_direction)
sigma_mu = np.std(data_direction)

print('The mean of mu is', mean_mu)
print('The sigma of mu is', sigma_mu)
print('The max value of mu is', max(data_direction))
print('The min value of mu is', min(data_direction))

mean_p = np.mean(data_polarization)
sigma_p = np.std(data_polarization)

print('The mean of p is', mean_p)
print('The sigma of p is', sigma_p)
print('The max value of p is', max(data_polarization))
print('The min value of p is', min(data_polarization))

# expected direction distribution - change y_fit to the corresponding expression (see next section)*
x_fit = np.arange(-1, 1, 0.001)
y_fit = -(3 * x_fit**2 - 9) / 16.

# plot - change the label to corresponding settings
plt.hist(data_direction, bins=1000, density=True, label='simulated')
plt.plot(x_fit, y_fit, label='theoretical')
plt.title('Probability density of mu (mu_i=0, p_i=0, x=8.63)')
plt.xlabel('Direction (mu)')
plt.ylabel('Probability density (P(mu))')
plt.legend(loc='upper right')
plt.show()

# analysis polarizations
polar0 = []
polar1 = []
polar2 = []
polar3 = []
polar4 = []
polar5 = []
polar6 = []
polar7 = []
polar8 = []
polar9 = []

for x in range(len(data_direction)):
    if data_direction[x] < -0.8:
        polar0.append(data_polarization[x])
    if data_direction[x] >= -0.8 and data_direction[x] < -0.6:
        polar1.append(data_polarization[x])
    if data_direction[x] >= -0.6 and data_direction[x] < -0.4:
        polar2.append(data_polarization[x])
    if data_direction[x] >= -0.4 and data_direction[x] < -0.2:
        polar3.append(data_polarization[x])
    if data_direction[x] >= -0.2 and data_direction[x] < 0.0:
        polar4.append(data_polarization[x])
    if data_direction[x] >= 0.0 and data_direction[x] < 0.2:
        polar5.append(data_polarization[x])
    if data_direction[x] >= 0.2 and data_direction[x] < 0.4:
        polar6.append(data_polarization[x])
    if data_direction[x] >= 0.4 and data_direction[x] < 0.6:
        polar7.append(data_polarization[x])
    if data_direction[x] >= 0.6 and data_direction[x] < 0.8:
        polar8.append(data_polarization[x])
    if data_direction[x] >= 0.8 and data_direction[x] < 1.0:
        polar9.append(data_polarization[x])
        
p0_avg = sum(polar0)/len(polar0)
p1_avg = sum(polar1)/len(polar1)
p2_avg = sum(polar2)/len(polar2)
p3_avg = sum(polar3)/len(polar3)
p4_avg = sum(polar4)/len(polar4)
p5_avg = sum(polar5)/len(polar5)
p6_avg = sum(polar6)/len(polar6)
p7_avg = sum(polar7)/len(polar7)
p8_avg = sum(polar8)/len(polar8)
p9_avg = sum(polar9)/len(polar9)
        
p0 = sum(polar0)/len(data_polarization)/0.2
p1 = sum(polar1)/len(data_polarization)/0.2
p2 = sum(polar2)/len(data_polarization)/0.2
p3 = sum(polar3)/len(data_polarization)/0.2
p4 = sum(polar4)/len(data_polarization)/0.2
p5 = sum(polar5)/len(data_polarization)/0.2
p6 = sum(polar6)/len(data_polarization)/0.2
p7 = sum(polar7)/len(data_polarization)/0.2
p8 = sum(polar8)/len(data_polarization)/0.2
p9 = sum(polar9)/len(data_polarization)/0.2

x2 = [-0.9, -0.7, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 0.7, 0.9]
y2 = [p0_avg, p1_avg, p2_avg, p3_avg, p4_avg, p5_avg, p6_avg, p7_avg, p8_avg, p9_avg]
y3 = [p0, p1, p2, p3, p4, p5, p6, p7, p8, p9]

# expected polarization distribution - change y_fit to the corresponding expression (see next section)*
x_p = np.arange(-1, 1, 0.01)
y_p = 3 * (1 - x_p**2) / 16.

plt.plot(x2, y2)
plt.title('Mean polarization in each direction bin')
plt.xlabel('Direction (mu)')
plt.ylabel('Average polarization(p)')
plt.show()

plt.plot(x2, y3)
plt.plot(x_p, y_p)
plt.title('Mean polarization in each direction bin')
plt.xlabel('Direction (mu)')
plt.ylabel('Probability density')
plt.show()

*expected direction with initial settings:
- pure diplo:

initial mu = 0:

initial p = 0, y_fit = -(3 * x_fit^2 - 9) / 16, y_p = 3 * (1 - x_p^2) / 16

initial p = 1, y_fit = 3 / 4 * (1 - x_fit^2), y_p = 3 / 4 * (1 - x_p^2)

initial p = -1, y_fit = 3 / 8 * (1 + x_fit^2), y_p = 3 / 8 * (x_p^2 - 1)

- mixed:

initial mu = 0:

initial p = 0, y_fit = 2 / 3 * 1 / 2 + 1 / 3 * ((9 - 3 * x_fit^2) / 16.)

initial p = 1, y_fit = 1 / 3 + 1 / 4 * (1 - x_fit^2)

initial p = -1, y_fit = 1 / 3 + 1 / 8 * (1 + x_fit^2)